# Homomorphic Encryption using Duet: Data Owner

### Setup

In [1]:
import syft as sy
import tenseal as ts
import pytest

sy.load("tenseal")

sy.load() is deprecated and not needed anymore


### Start Duet Instance

In [2]:
duet = sy.launch_duet(loopback=True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...

♫♫♫ > CONNECTED!



### TenSeal Context

In [3]:
context = ts.Context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.global_scale = 2**40
context

### Encrypt the Data

In [4]:
v1 = [0, 1, 2, 3, 4]
v2 = [4, 3, 2, 1, 0]

enc_v1 = ts.ckks_vector(context, v1)
enc_v2 = ts.ckks_vector(context, v2)
(enc_v1, enc_v2)

(<tenseal.tensors.ckksvector.CKKSVector at 0x1b81636c7f0>,
 <tenseal.tensors.ckksvector.CKKSVector at 0x1b81636c580>)

In [5]:
result_add_local = enc_v1 + enc_v2
result_add_local.decrypt()

[4.00000000070795,
 4.000000000608912,
 3.9999999984717802,
 3.9999999978585508,
 4.000000000015286]

### Make Context and Encrypted Vectors Referenceable over Duet

In [6]:
# tag them so our partner can easily reference it
ctx_ptr = context.send(duet, pointable=True, tags=["context"])
enc_v1_ptr = enc_v1.send(duet, pointable=True, tags=["enc_v1"])
enc_v2_ptr = enc_v2.send(duet, pointable=True, tags=["enc_v2"])

In [7]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: a88d63b923c848d9bf4f8c0858f7034a>,[context],,<class 'tenseal.enc_context.Context'>
1,<UID: f1cd6e1fd38b4341ab79158011acc2f8>,[enc_v1],,<class 'tenseal.tensors.ckksvector.CKKSVector'>
2,<UID: a36bb89b93e346e2b05f1d03088eb5fd>,[enc_v2],,<class 'tenseal.tensors.ckksvector.CKKSVector'>


###  check requests

In [11]:
duet.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,[context],I would like to get the context,<UID: ff99b91b80ea44f39d7505aa82a4ae8b>,<UID: a88d63b923c848d9bf4f8c0858f7034a>,<class 'tenseal.enc_context.Context'>
1,[enc_v1],I would like to get first vector,<UID: ec2cf58d73d54a42930a5fe23752a5cc>,<UID: f1cd6e1fd38b4341ab79158011acc2f8>,<class 'tenseal.tensors.ckksvector.CKKSVector'>
2,[enc_v2],I would like to get second vector,<UID: b972ea0db96645418568397b4efc0fef>,<UID: a36bb89b93e346e2b05f1d03088eb5fd>,<class 'tenseal.tensors.ckksvector.CKKSVector'>


### Approve the requests

In [12]:
duet.requests[0].accept()
duet.requests[0].accept()
duet.requests[0].accept()

In [13]:
duet.requests.pandas

""


### Get the computation results from store and decrypt them locally

In [15]:
# Validate the encrypted add
result_add = duet.store["result_add"].get(delete_obj=False)
result_add.link_context(context)

result_add

In [16]:
decrypted_result = result_add.decrypt()
regular_add = [v1 + v2 for v1, v2 in zip(v1, v2)]
assert pytest.approx(decrypted_result, abs=10**-3) == regular_add

decrypted_result = [round(r, 9) for r in decrypted_result]
print(f"Regular addition: {regular_add}\n"
      f"Decrypted addition: {decrypted_result}  ")

Regular addition: [4, 4, 4, 4, 4]
Decrypted addition: [4.000000001, 4.000000001, 3.999999998, 3.999999998, 4.0]  


In [17]:
# Validate the encrypted - plain add
result_iadd = duet.store["result_iadd"].get(delete_obj=False)
result_iadd.link_context(context)

decrypted_result = result_iadd.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [v1 + v2 for v1, v2 in zip(v1, [10, 10, 10, 10, 10])]

decrypted_result

[9.999999999902128,
 10.999999999105661,
 11.999999999949662,
 12.999999998883103,
 14.000000000068258]

In [18]:
# Validate the encrypted subtraction

result_sub = duet.store["result_sub"].get(delete_obj=False)
result_sub.link_context(context)

decrypted_result = result_sub.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [v1 - v2 for v1, v2 in zip(v1, v2)]

decrypted_result

[-4.000000000889013,
 -2.000000002388621,
 1.334405475006406e-09,
 1.9999999999515825,
 4.000000000030301]

In [19]:
# Validate the encrypted multiplication

result_mul = duet.store["result_mul"].get(delete_obj=False)
result_mul.link_context(context)

decrypted_result = result_mul.decrypt()
regular_mult = [v1 * v2 for v1, v2 in zip(v1, v2)]
assert pytest.approx(decrypted_result, abs=10**-3) == regular_mult

decrypted_result = [round(r, 9) for r in decrypted_result]

print(f"Regular multiplication: {regular_mult}\n"
      f"Decrypted multiplication: {decrypted_result}")

Regular multiplication: [0, 3, 4, 3, 0]
Decrypted multiplication: [0.0, 3.000000399, 4.000000535, 3.000000402, -0.0]


In [20]:
# Validate the encrypted power

result_pow = duet.store["result_pow"].get(delete_obj=False)
result_pow.link_context(context)

decrypted_result = result_pow.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [v ** 3 for v in v1]

decrypted_result

[-3.4185454467206e-10,
 1.000000800606717,
 8.000006436454456,
 27.00002169492351,
 64.00005149951267]

In [21]:
# Validate the encrypted negation

result_neg = duet.store["result_neg"].get(delete_obj=False)
result_neg.link_context(context)

decrypted_result = result_neg.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [-v for v in v1]

decrypted_result

[9.053202632003376e-11,
 -0.999999999110146,
 -1.9999999999030922,
 -2.999999998905067,
 -4.000000000022794]

In [22]:
# Validate the encrypted polynomial evaluation for 1 + X^2 + X^3

result_poly = duet.store["result_poly"].get(delete_obj=False)
result_poly.link_context(context)

decrypted_result = result_poly.decrypt()
regular_poly = [1 + v**2 + v**3 for v in v1]
assert pytest.approx(decrypted_result, abs=10**-3) == regular_poly

decrypted_result = [round(r, 9) for r in decrypted_result]

print(f"Regular polynom: {regular_poly}\n"
      f"Decrypted polynom: {decrypted_result}")

Regular polynom: [1, 3, 13, 37, 81]
Decrypted polynom: [0.999999999, 3.000000933, 13.00000697, 37.000022897, 81.000053645]
